**SPEECH EMOTION DETECTION USING MLP**

> Indented block



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}


In [3]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [4]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/content/drive/My Drive/wav/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          y.append(emotion)
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [5]:
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score

import os
import pickle

In [6]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])
# best model, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

# now we save the model
# make result directory if doesn't exist yet
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

03-01-07-02-01-01-22.wav
03-01-07-01-01-02-22.wav
03-01-07-01-02-01-22.wav
03-01-03-02-01-01-22.wav
03-01-03-01-02-01-22.wav
03-01-05-01-01-01-22.wav
03-01-02-02-02-02-22.wav
03-01-05-01-02-02-22.wav
03-01-02-01-01-02-22.wav
03-01-06-01-01-02-22.wav
03-01-05-02-02-02-22.wav
03-01-04-01-01-02-22.wav
03-01-03-02-02-02-22.wav
03-01-06-02-02-01-22.wav
03-01-04-02-02-01-22.wav
03-01-05-02-02-01-22.wav
03-01-06-02-02-02-22.wav
03-01-02-01-02-01-22.wav
03-01-02-02-01-02-22.wav
03-01-07-01-02-02-22.wav
03-01-06-01-02-02-22.wav
03-01-05-02-01-02-22.wav
03-01-04-01-01-01-22.wav
03-01-03-01-01-01-22.wav
03-01-02-01-01-01-22.wav
03-01-01-01-02-02-22.wav
03-01-06-01-02-01-22.wav
03-01-07-02-02-01-22.wav
03-01-06-01-01-01-22.wav
03-01-07-01-01-01-22.wav
03-01-06-02-01-01-22.wav
03-01-04-02-01-01-22.wav
03-01-06-02-01-02-22.wav
03-01-03-01-01-02-22.wav
03-01-01-01-01-01-22.wav
03-01-01-01-01-02-22.wav
03-01-05-02-01-01-22.wav
03-01-04-02-02-02-22.wav
03-01-03-02-01-02-22.wav
03-01-03-02-02-01-22.wav


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


Accuracy: 63.16%


In [7]:
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

[*] Training the model...


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


Accuracy: 63.16%


In [8]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.60      0.60      0.60         5
       happy       0.50      0.80      0.62         5
     neutral       1.00      0.67      0.80         3
         sad       0.75      0.50      0.60         6

    accuracy                           0.63        19
   macro avg       0.71      0.64      0.65        19
weighted avg       0.68      0.63      0.64        19

[[3 2 0 0]
 [1 4 0 0]
 [0 0 2 1]
 [1 2 0 3]]


**Second**

In [9]:
m_params = {
    'alpha': 0.01,
    'batch_size': 200,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
m1 = MLPClassifier(**m_params)

# train the model
print("[*] Training the model...")
m1.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_p = m1.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_p)

print("Accuracy: {:.2f}%".format(accuracy*100))

# now we save the model
# make result directory if doesn't exist yet

[*] Training the model...


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


Accuracy: 78.95%


In [10]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
print(classification_report(y_test,y_p))
print(confusion_matrix(y_test,y_p))

              precision    recall  f1-score   support

       angry       1.00      0.80      0.89         5
       happy       0.62      1.00      0.77         5
     neutral       1.00      0.67      0.80         3
         sad       0.80      0.67      0.73         6

    accuracy                           0.79        19
   macro avg       0.86      0.78      0.80        19
weighted avg       0.84      0.79      0.79        19

[[4 1 0 0]
 [0 5 0 0]
 [0 0 2 1]
 [0 2 0 4]]


**Third**

In [11]:
mn_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,2), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
m2 = MLPClassifier(**mn_params)

# train the model
print("[*] Training the model...")
m2.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_p2 = m2.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_p2)

print("Accuracy: {:.2f}%".format(accuracy*100))

[*] Training the model...


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


Accuracy: 26.32%


In [12]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
print(classification_report(y_test,y_p2))
print(confusion_matrix(y_test,y_p2))

              precision    recall  f1-score   support

       angry       0.00      0.00      0.00         5
       happy       0.26      1.00      0.42         5
     neutral       0.00      0.00      0.00         3
         sad       0.00      0.00      0.00         6

    accuracy                           0.26        19
   macro avg       0.07      0.25      0.10        19
weighted avg       0.07      0.26      0.11        19

[[0 5 0 0]
 [0 5 0 0]
 [0 3 0 0]
 [0 6 0 0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
import os
import wave
import pickle
from sys import byteorder
from array import array
from struct import pack
from sklearn.neural_network import MLPClassifier
import librosa


In [14]:
#print("Please talk")
#filename = "D:\project_emotion/test2.wav"# record the file (start t
filename = '/content/drive/My Drive/path.wav'
    #record_to_file(filename)
    # extract features and reshape it
features = extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1)
    # predict
result = model.predict(features)[0]
    # show the result !
print("result:", result)

result: angry
